In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.custom_generator_NEW_3 import CustomGeneratorNEW3
from tohu.utils import print_generated_sequence

## Custom generator without `__init__` method

The simplest way to use `CustomGeneratorNEW3` is with only class attributes (without `__init__` method).

In [2]:
class QuuxGenerator(CustomGeneratorNEW3):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    cc = FakerGenerator(method="name")

In [3]:
g = QuuxGenerator()

In [4]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


Note that while the _instance_ `g` contains attributes with the same names `"aa"` and `"bb"` as the parent class `QuuxGenerator`, internally the instance attributes are actually spawned versions of the class attributes (in particular, they are not identical):

In [5]:
print(QuuxGenerator.aa)
print(g.aa)
assert QuuxGenerator.aa is not g.aa

print(QuuxGenerator.bb)
print(g.bb)
assert QuuxGenerator.bb is not g.bb

print(QuuxGenerator.cc)
print(g.cc)
assert QuuxGenerator.cc is not g.cc

<Integer (id=0f20cf)>
<Integer (id=9f1efa)>
<HashDigest (id=8a6d63)>
<HashDigest (id=322f6b)>
<FakerGenerator (id=d91cee)>
<FakerGenerator (id=d6a9d9)>


## Custom generator with `__init__` method

Tohu generators that are defined as instance attributes within the `__init__` method are also picked up.

In [6]:
class QuuxGenerator(CustomGeneratorNEW3):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    
    def __init__(self, method):
        self.cc = FakerGenerator(method=method)

In [7]:
g = QuuxGenerator(method="first_name")

In [8]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Mary')
Quux(aa=171, bb='54596E', cc='Jennifer')
Quux(aa=142, bb='2A16D0', cc='Christopher')
Quux(aa=140, bb='FDCDD3', cc='Joseph')
Quux(aa=121, bb='BDE283', cc='Kelly')


## Naming requirements

The name of the custom generator class being defined must end with `[...]Generator` (so that tohu can automatically devise the name of tohu items produced by the generator). If this is not the case an error is raised when the class is instantiated.

In [9]:
class QuuxGen(CustomGeneratorNEW3):
    aa = Integer(100, 200)

In [10]:
import pytest

with pytest.raises(ValueError, match="Name of custom generator class must end with '\[...\]Generator', got: 'QuuxGen'"):
    _ = QuuxGen()

## Custom generators with loop variables as external dependencies

Custom generator provides a classmethod to register loop variables with the class. Once an instance of the custom generator class is instantiated, the method `extract_loop_runner()` returns a loop runner containing spawns of the previously registered loop variables. This is used internally to implement looped custom generators.

In [11]:
from tohu.loop_variable_NEW_3 import LoopVariableNEW3
from tohu.derived_generators import Apply

In [12]:
xx = LoopVariableNEW3("xx", values=[111, 222, 333]).set_loop_level(42)
xx_spawned_1 = xx.spawn()
xx_spawned_2 = xx.spawn()

In [13]:
class QuuxGenerator(CustomGeneratorNEW3):
    aa = Integer(100, 200)
    bb = Apply(lambda a: a*a, xx)

QuuxGenerator.set_dependency_mapping_for_next_instance_creation({xx: xx_spawned_1})

In [14]:
#QuuxGenerator._tohu_cg_class_loop_variables

In [15]:
g1 = QuuxGenerator()

In [16]:
#loop_runner = g.extract_loop_runner()

In [17]:
#loop_runner.loop_variables

In [18]:
xx_spawned_1.rewind_loop_variable()
print_generated_sequence(g1, num=5, sep="\n", seed=11111)

print("\n---\n")

xx_spawned_1.advance()
print_generated_sequence(g1, num=5, sep="\n", seed=11111)

Generated sequence:

Quux(aa=163, bb=12321)
Quux(aa=171, bb=12321)
Quux(aa=142, bb=12321)
Quux(aa=140, bb=12321)
Quux(aa=121, bb=12321)

---

Generated sequence:

Quux(aa=163, bb=49284)
Quux(aa=171, bb=49284)
Quux(aa=142, bb=49284)
Quux(aa=140, bb=49284)
Quux(aa=121, bb=49284)


In [19]:
QuuxGenerator.set_dependency_mapping_for_next_instance_creation({xx: xx_spawned_2})
g2 = QuuxGenerator()

xx_spawned_2.rewind_loop_variable()
print_generated_sequence(g2, num=5, sep="\n", seed=22222)
print("\n---\n")

xx_spawned_2.advance()
print_generated_sequence(g2, num=5, sep="\n")
print("\n---\n")

xx_spawned_1.rewind_loop_variable()
print_generated_sequence(g1, num=5, sep="\n", seed=11111)
print("\n---\n")

xx_spawned_2.advance()
print_generated_sequence(g2, num=5, sep="\n")

Generated sequence:

Quux(aa=108, bb=12321)
Quux(aa=166, bb=12321)
Quux(aa=164, bb=12321)
Quux(aa=163, bb=12321)
Quux(aa=156, bb=12321)

---

Generated sequence:

Quux(aa=145, bb=49284)
Quux(aa=170, bb=49284)
Quux(aa=114, bb=49284)
Quux(aa=170, bb=49284)
Quux(aa=167, bb=49284)

---

Generated sequence:

Quux(aa=163, bb=12321)
Quux(aa=171, bb=12321)
Quux(aa=142, bb=12321)
Quux(aa=140, bb=12321)
Quux(aa=121, bb=12321)

---

Generated sequence:

Quux(aa=153, bb=110889)
Quux(aa=172, bb=110889)
Quux(aa=136, bb=110889)
Quux(aa=194, bb=110889)
Quux(aa=153, bb=110889)
